In [2]:
import pandas as pd
import numpy as np
import os
import time
from scipy.stats import norm, lognorm, laplace


In [3]:
dfa = pd.read_csv('/gd/bayesian_RT/Alignments/FP18_20180801_2/ev_updated.txt', sep='\t', low_memory=False)
dfa

,Sequence,Length,Modifications,Modified sequence,Oxidation (M) Probabilities,Oxidation (M) Score Diffs,Acetyl (Protein N-term),Oxidation (M),Missed cleavages,Proteins,...,muij,pep_new,peptide_id,rt_minus,rt_plus,sigmaij,stan_peptide_id,pep_updated,q-value,residual
0,AAAAAAALQAK,11,Unmodified,_AAAAAAALQAK_,NaN,NaN,0,0,0,sp|P36578|RL4_HUMAN60SribosomalproteinL4OS=Hom...,...,33.297842,9.344208e-11,0.0,0.039736,5.267562e+00,0.066777,0.0,9.344208e-11,1.237011e-11,0.010158
1,AAAAAAALQAK,11,Unmodified,_AAAAAAALQAK_,NaN,NaN,0,0,0,sp|P36578|RL4_HUMAN60SribosomalproteinL4OS=Hom...,...,33.232577,1.976120e-06,0.0,0.037944,6.074239e+00,0.058429,0.0,1.976120e-06,2.929099e-07,-0.016577
2,AAAAAAALQAK,11,Unmodified,_AAAAAAALQAK_,NaN,NaN,0,0,0,sp|P36578|RL4_HUMAN60SribosomalproteinL4OS=Hom...,...,33.232577,1.047489e-07,0.0,0.037944,6.575127e+00,0.058429,0.0,1.047489e-07,1.302698e-08,-0.009577
3,AAAAAAALQAK,11,Unmodified,_AAAAAAALQAK_,NaN,NaN,0,0,0,sp|P36578|RL4_HUMAN60SribosomalproteinL4OS=Hom...,...,32.916888,2.104331e-06,0.0,0.038275,5.754051e+00,0.058496,0.0,2.104331e-06,3.148373e-07,-0.005888
4,AAAAAAALQAK,11,Unmodified,_AAAAAAALQAK_,NaN,NaN,0,0,0,sp|P36578|RL4_HUMAN60SribosomalproteinL4OS=Hom...,...,32.916888,9.103231e-07,0.0,0.038276,6.168647e+00,0.058496,0.0,9.103231e-07,1.295563e-07,0.000112
5,AAAAAAALQAK,11,Unmodified,_AAAAAAALQAK_,NaN,NaN,0,0,0,sp|P36578|RL4_HUMAN60SribosomalproteinL4OS=Hom...,...,32.887191,9.207835e-11,0.0,0.037871,1.958994e+00,0.087139,0.0,9.207835e-11,1.227428e-11,0.086809
6,AAAAAAALQAK,11,Unmodified,_AAAAAAALQAK_,NaN,NaN,0,0,0,sp|P36578|RL4_HUMAN60SribosomalproteinL4OS=Hom...,...,32.887191,1.747583e-05,0.0,0.037873,2.122856e+00,0.087139,0.0,1.747583e-05,3.050117e-06,0.079809
7,AAAAAK,6,Unmodified,_AAAAAK_,NaN,NaN,0,0,0,sp|A6NHY2|AKD1B_HUMANAnkyrinrepeatanddeathdoma...,...,20.999932,3.293274e-03,1.0,0.017765,3.143359e+00,0.061636,1.0,3.293274e-03,3.309283e-04,0.060068
8,AAAAAK,6,Unmodified,_AAAAAK_,NaN,NaN,0,0,0,sp|A6NHY2|AKD1B_HUMANAnkyrinrepeatanddeathdoma...,...,21.055742,5.889593e-04,1.0,0.017592,4.144867e+00,0.058330,1.0,5.889593e-04,1.024246e-04,0.032258
9,AAAAAK,6,Unmodified,_AAAAAK_,NaN,NaN,0,0,0,sp|A6NHY2|AKD1B_HUMANAnkyrinrepeatanddeathdoma...,...,21.064028,3.009750e-04,1.0,0.018915,6.777428e+00,0.050763,1.0,3.009750e-04,5.441892e-05,-0.000028


In [5]:
params = {}
params['exp'] = pd.read_csv('/gd/bayesian_RT/Alignments/FP18_20180730_1/exp_params.txt', sep='\t')
params['peptide'] = pd.read_csv('/gd/bayesian_RT/Alignments/FP18_20180730_1/peptide_params.txt', sep='\t')
params['pair'] = pd.read_csv('/gd/bayesian_RT/Alignments/FP18_20180730_1/pair_params.txt', sep='\t')
params

{'exp':       beta_0    beta_1    beta_2  sigma_intercept  sigma_slope  split_point  \
 0   0.274781  0.991910  0.948970         0.077035     0.092141    31.607452   
 1   0.019658  1.007379  0.988126         0.060941     0.083515    36.749747   
 2  -0.064145  1.010244  0.985038         0.056180     0.080774    34.951608   
 3  -0.081333  1.006814  0.989877         0.045506     0.073505    35.119617   
 4  -0.210192  1.011077  0.990294         0.036826     0.066223    34.548450   
 5  -0.236744  1.009134  0.995901         0.046165     0.073517    28.950596   
 6  -0.248827  1.012361  0.991026         0.043377     0.071054    30.780129   
 7  -0.072532  1.007985  0.983111         0.043083     0.071569    39.676987   
 8  -0.253821  1.012992  0.997271         0.037389     0.063552    29.943424   
 9  -0.209785  1.000624  0.988291         0.036659     0.065962    36.113367   
 10 -0.332659  1.000506  1.020883         0.059040     0.084875    28.342319   
 
     exp_id  init_beta_0  init_

In [9]:
dff = dfa[dfa['exclude']==False]
dff = dff.reset_index(drop=True)

#logger.info('{} / {} ({:.2%}) confident, alignable observations (PSMs) after filtering.'.format(dff.shape[0], dfa.shape[0], dff.shape[0] / dfa.shape[0]))

# refactorize peptide id into stan_peptide_id, 
# to preserve continuity when feeding data into STAN
dff['stan_peptide_id'] = dff['peptide_id'].map({
ind: val for val, ind in enumerate(dff['peptide_id'].unique())})

num_experiments = dff['exp_id'].max() + 1
num_observations = dff.shape[0]
num_peptides = dff['peptide_id'].max() + 1
exp_names = np.sort(dff['Raw file'].unique())
mean_log_rt = np.mean(np.log(dff['Retention time']))
sd_log_rt = np.std(np.log(dff['Retention time']))
max_rt = dff['Retention time'].max()
pep_id_list = dff['peptide_id'].unique()

dff

,Sequence,Length,Modifications,Modified sequence,Oxidation (M) Probabilities,Oxidation (M) Score Diffs,Acetyl (Protein N-term),Oxidation (M),Missed cleavages,Proteins,...,muij,pep_new,peptide_id,rt_minus,rt_plus,sigmaij,stan_peptide_id,pep_updated,q-value,residual
0,AAAAAAALQAK,11,Unmodified,_AAAAAAALQAK_,NaN,NaN,0,0,0,sp|P36578|RL4_HUMAN60SribosomalproteinL4OS=Hom...,...,33.297842,9.344208e-11,0.0,0.039736,5.267562,0.066777,0,9.344208e-11,1.237011e-11,0.010158
1,AAAAAAALQAK,11,Unmodified,_AAAAAAALQAK_,NaN,NaN,0,0,0,sp|P36578|RL4_HUMAN60SribosomalproteinL4OS=Hom...,...,33.232577,1.976120e-06,0.0,0.037944,6.074239,0.058429,0,1.976120e-06,2.929099e-07,-0.016577
2,AAAAAAALQAK,11,Unmodified,_AAAAAAALQAK_,NaN,NaN,0,0,0,sp|P36578|RL4_HUMAN60SribosomalproteinL4OS=Hom...,...,33.232577,1.047489e-07,0.0,0.037944,6.575127,0.058429,0,1.047489e-07,1.302698e-08,-0.009577
3,AAAAAAALQAK,11,Unmodified,_AAAAAAALQAK_,NaN,NaN,0,0,0,sp|P36578|RL4_HUMAN60SribosomalproteinL4OS=Hom...,...,32.916888,2.104331e-06,0.0,0.038275,5.754051,0.058496,0,2.104331e-06,3.148373e-07,-0.005888
4,AAAAAAALQAK,11,Unmodified,_AAAAAAALQAK_,NaN,NaN,0,0,0,sp|P36578|RL4_HUMAN60SribosomalproteinL4OS=Hom...,...,32.916888,9.103231e-07,0.0,0.038276,6.168647,0.058496,0,9.103231e-07,1.295563e-07,0.000112
5,AAAAAAALQAK,11,Unmodified,_AAAAAAALQAK_,NaN,NaN,0,0,0,sp|P36578|RL4_HUMAN60SribosomalproteinL4OS=Hom...,...,32.887191,9.207835e-11,0.0,0.037871,1.958994,0.087139,0,9.207835e-11,1.227428e-11,0.086809
6,AAAAAAALQAK,11,Unmodified,_AAAAAAALQAK_,NaN,NaN,0,0,0,sp|P36578|RL4_HUMAN60SribosomalproteinL4OS=Hom...,...,32.887191,1.747583e-05,0.0,0.037873,2.122856,0.087139,0,1.747583e-05,3.050117e-06,0.079809
7,AAAAAK,6,Unmodified,_AAAAAK_,NaN,NaN,0,0,0,sp|A6NHY2|AKD1B_HUMANAnkyrinrepeatanddeathdoma...,...,20.999932,3.293274e-03,1.0,0.017765,3.143359,0.061636,1,3.293274e-03,3.309283e-04,0.060068
8,AAAAAK,6,Unmodified,_AAAAAK_,NaN,NaN,0,0,0,sp|A6NHY2|AKD1B_HUMANAnkyrinrepeatanddeathdoma...,...,21.055742,5.889593e-04,1.0,0.017592,4.144867,0.058330,1,5.889593e-04,1.024246e-04,0.032258
9,AAAAAK,6,Unmodified,_AAAAAK_,NaN,NaN,0,0,0,sp|A6NHY2|AKD1B_HUMANAnkyrinrepeatanddeathdoma...,...,21.064028,3.009750e-04,1.0,0.018915,6.777428,0.050763,1,3.009750e-04,5.441892e-05,-0.000028


In [243]:
np.array([1,2,3,4,4])[[2, 3, -1]]
np.sum(pd.isnull(dfa['stan_peptide_id']))


pandas.core.series.Series

In [25]:
dfa[-((dfa['PEP'] > 0.1) & (dfa['PEP'] < 0.8))]

,Sequence,Length,Modifications,Modified sequence,Oxidation (M) Probabilities,Oxidation (M) Score Diffs,Acetyl (Protein N-term),Oxidation (M),Missed cleavages,Proteins,...,exp_id,mu,muij,pep_new,peptide_id,rt_minus,rt_plus,sigmaij,stan_peptide_id,pep_updated
0,AAAAAAAAAAGAAGGR,16,Acetyl (Protein N-term),_(ac)AAAAAAAAAAGAAGGR_,NaN,NaN,1,0,0,sp|Q86U42-2|PABP2_HUMANIsoform2ofPolyadenylate...,...,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,AAAAAAAAAAGAAGGR,16,Acetyl (Protein N-term),_(ac)AAAAAAAAAAGAAGGR_,NaN,NaN,1,0,0,sp|Q86U42-2|PABP2_HUMANIsoform2ofPolyadenylate...,...,8.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,AAAAAAAAAAGAAGGR,16,Acetyl (Protein N-term),_(ac)AAAAAAAAAAGAAGGR_,NaN,NaN,1,0,0,sp|Q86U42-2|PABP2_HUMANIsoform2ofPolyadenylate...,...,9.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,AAAAAAAAAAGAAGGR,16,Acetyl (Protein N-term),_(ac)AAAAAAAAAAGAAGGR_,NaN,NaN,1,0,0,sp|Q86U42-2|PABP2_HUMANIsoform2ofPolyadenylate...,...,16.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,AAAAAAAAAAGAAGGR,16,Acetyl (Protein N-term),_(ac)AAAAAAAAAAGAAGGR_,NaN,NaN,1,0,0,sp|Q86U42-2|PABP2_HUMANIsoform2ofPolyadenylate...,...,19.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
5,AAAAAAAAAAGAAGGR,16,Acetyl (Protein N-term),_(ac)AAAAAAAAAAGAAGGR_,NaN,NaN,1,0,0,sp|Q86U42-2|PABP2_HUMANIsoform2ofPolyadenylate...,...,25.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
6,AAAAAAAAAAGAAGGR,16,Acetyl (Protein N-term),_(ac)AAAAAAAAAAGAAGGR_,NaN,NaN,1,0,0,sp|Q86U42-2|PABP2_HUMANIsoform2ofPolyadenylate...,...,27.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
7,AAAAAAAAAAR,11,Unmodified,_AAAAAAAAAAR_,NaN,NaN,0,0,0,sp|Q9Y651|SOX21_HUMANTranscriptionfactorSOX-21...,...,15.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,8.176700e-01
8,AAAAAAALQAK,11,Unmodified,_AAAAAAALQAK_,NaN,NaN,0,0,0,sp|P36578|RL4_HUMAN60SribosomalproteinL4OS=Hom...,...,1.0,35.116931,37.365363,1.264768e-05,2.0,0.047270,1.568082e-01,0.320521,0.0,1.264768e-05
9,AAAAAAALQAK,11,Unmodified,_AAAAAAALQAK_,NaN,NaN,0,0,0,sp|P36578|RL4_HUMAN60SribosomalproteinL4OS=Hom...,...,2.0,35.116931,33.131558,1.510565e-05,2.0,0.064123,1.362949e-01,0.243548,0.0,1.510565e-05


In [41]:
seqs = dfa['Sequence'].unique()
seqs = seqs[np.random.randint(0, len(seqs), 100)]
seqs

array(['EDQTEYLEER', 'QPRPVR', 'MAYQSLR', 'FGGGNPELLTQMVSK', 'EQIESPLELK',
       'TASAGK', 'FVLSQAKDEL', 'ELQEIMAIDMDQK', 'NKESKDPADETEAD',
       'VTHFPAWDTR', 'QDSLVINLNR', 'SVASVK', 'DSPWGPTGTR', 'VAGGQLHAGK',
       'ASLQTRMEFSKWDEEK', 'QRLDYEAQLR',
       'SLLGGHYDPGFMATSPPEDRPGGGGGAAGGAEDLAELDQLLR', 'ALHTPQR', 'AYIDSFR',
       'REPAEQPGDGER', 'RPAGHR', 'VPGPRR', 'EAIIDAK', 'VDKKVEPK',
       'ETGLNAVLFMDRGIAMCSR', 'VLNTNIDGR', 'ATGVRGLEKIAK', 'TGAVPK',
       'ESEGLPQPEQK', 'AGARASWMPMFQR', 'VGRHLDTFPTEADRQR', 'SQGHKDK',
       'IESTKNEQLR', 'YQLSSEAAK', 'YQQLFEDIR', 'FEEEAR', 'TSSFIR',
       'DGLHRHLQQTRYSVPDTNVVK', 'DHMNYRHIVDK', 'MQIHFK', 'DAYRFIPHDGCK',
       'KAADEAK', 'SRGCEAPYVNAK', 'DDKTIIEEQATK', 'LGGFLK', 'LPRPLR',
       'VLSRGLR', 'NIFVKEEQELPPK', 'EQSSSSQSSK', 'LLVEFR', 'MVQEAEK',
       'EAYMGNVLQGGEGQAPTR', 'VDNIIK', 'ALAPEYAK', 'INYTEGR', 'VLNTNIDGR',
       'MIYASSKDAIK', 'LLTELK', 'MSVRGK', 'RLVGGLPFK', 'GGASELQEDESFTLR',
       'VLEDSDLK', 'PRIFFLK', 'ITSVE

In [219]:
dfe = ((dfa['Sequence'].isin(seqs)) & (dfa['exp_id'] != 0))
#dfe = dfa.loc[dfe].groupby('Sequence')[['PEP', 'Retention time']].apply(lambda x: (x['Retention time'].values.tolist(),
#                                                                                   x['PEP'].values.tolist()))
#dfe = dfa.loc[dfe].groupby('Sequence')[['Retention time']].apply(lambda x: (x['Retention time'].values))
dfe = dfa.loc[dfe]\
.groupby('Sequence')['Retention time'].apply(lambda x: x.values).values.tolist()
#dfe.apply(np.array)
#dfe.sort_index()
#dfe
[len(e) for e in dfe]
dfe

[array([29.108, 24.511, 24.519, 32.891, 32.871, 32.152, 31.121, 31.251,
        32.723, 32.817, 32.066, 31.628, 31.817, 29.599, 31.846, 32.035,
        38.012, 37.906]),
 array([37.127, 37.124, 36.988, 35.542, 35.571, 33.571, 33.488, 33.399,
        33.457, 32.69 , 32.305, 32.525, 32.456, 32.602, 29.855, 29.86 ,
        29.963, 29.872, 29.672, 30.234, 35.988, 32.278, 37.127, 37.124,
        36.662, 36.988, 35.113, 35.233, 35.542, 35.253, 35.571, 26.283,
        33.571, 33.488, 33.399, 33.457, 32.69 , 33.306, 32.237, 33.499,
        30.649, 31.121, 33.298, 32.305, 33.616, 32.525, 32.456, 32.602,
        33.323, 33.29 , 29.855, 30.445, 29.86 , 30.492, 30.275, 29.963,
        30.597, 32.919, 32.928, 31.798, 32.574, 31.905, 32.656, 29.872,
        29.672, 30.971, 39.768, 39.503]),
 array([35.784, 32.144, 37.522, 37.403, 35.781, 35.841, 35.871, 35.879,
        30.577, 35.015, 25.907, 25.924, 34.801, 34.773, 34.639, 34.677,
        33.834, 34.42 , 33.341, 33.539, 31.474, 31.152, 31.137, 33.8

In [163]:
dfe.apply(lambda x: len(x[0])).max()
#mu_k = pd.DataFrame(np.random.rand(100), index=seqs)
#mu_k

88

In [169]:
start=time.time()

for i in range(0, 100):
    for j in range(0, 1000):
        np.random.randint(0, 15, 15)

print('{:.3f} ms'.format((time.time()-start) * 1000))

start=time.time()

for i in range(0, 100):
    for j in range(0, 500):
        i*j
np.random.rand(5000000)

print('{:.3f} ms'.format((time.time()-start) * 1000))

239.510 ms
75.765 ms


[1.2, 1.3, 1.5, 2.1]

In [241]:
arr = np.array([1, 1, 3, 4, 10, 10])
{ind: var for var, ind in enumerate(np.unique(arr))}

{1: 0, 3: 1, 4: 2, 10: 3}

In [14]:
seqs = dfa['peptide_id'].unique()
seqs = seqs[np.random.randint(0, len(seqs), 100)]
seqs
dfe = ((dfa['peptide_id'].isin(seqs)) & (dfa['exp_id'] != 0))
dfe = dfa.loc[dfe, ['stan_peptide_id', 'PEP', 'mu', 'sigmaij', 'exp_id']]
dfe

,stan_peptide_id,PEP,mu,sigmaij,exp_id
87,9.0,0.121510,19.870919,0.060112,3.0
88,9.0,0.309750,19.870919,0.060774,5.0
89,9.0,0.213410,19.870919,0.057496,6.0
90,9.0,0.015316,19.870919,0.050018,8.0
649,54.0,0.006216,36.945303,0.086022,2.0
650,54.0,0.039902,36.945303,0.072663,3.0
651,54.0,0.003985,36.945303,0.061292,4.0
652,54.0,0.009452,36.945303,0.073326,5.0
653,54.0,0.009956,36.945303,0.069628,6.0
654,54.0,0.003218,36.945303,0.069525,7.0


In [16]:
np.apply_along_axis(np.sum, 1, np.array([[0, 1], [2, 3]]))

array([1, 5])

In [96]:
from scipy.stats import bernoulli, binom, uniform
import time

In [48]:
_time = time.time()
[bernoulli.rvs(p, size=1000).astype(bool) for p in np.random.rand(100)]
print(time.time() - _time)

_time = time.time()
probs = np.random.rand(100)
for i in range(0, 100):
    bernoulli.rvs(probs[i], size=1000).astype(bool)
print(time.time() - _time)

0.023654937744140625
0.016099214553833008


In [167]:
A = np.random.rand(1000, 1000)

_time = time.time()
np.apply_along_axis(np.median, 1, A)
print(time.time() - _time)

_time = time.time()
for i in range(0, A.shape[0]):
    np.median(A[i,:])
print(time.time() - _time)

#np.median([0, 1, 0, 1], out=A[0,0])
np.median([[1, 2], [3, 3], [1, 3]])

A[[],1] = []

0.07963991165161133
0.05855393409729004


In [95]:
_time = time.time()
for i in range(0, 1000):
    for j in range(0, 100):
        bernoulli.rvs(0.1, size=100)
print(time.time() - _time)

5.659702777862549


In [128]:
_time = time.time()
pool = uniform.rvs(size=1000*100*100)
counter = 0
for i in range(0, 1000):
    for j in range(0, 100):
        fp = (pool[counter:(counter+100)] < 0.1).astype(bool)
        counter += 100
        if fp.sum() == 0: continue
print(time.time() - _time)
print(counter)

0.7620499134063721
10000000


In [111]:
_time = time.time()
for i in range(0, 1000):
    for j in range(0, 100):
        norm.rvs(loc=0, scale=2, size=10)
print(time.time() - _time)

7.933864116668701


In [163]:
_time = time.time()
#null_dists = [norm(loc=i, scale=2) for i in np.random.rand(10)]
#null_dists = np.array(null_dists)
null_dists = np.array([np.random.rand(10), np.random.rand(10)]).T

exp_ids = (np.random.rand(100)*10).astype(int)

for i in range(0, 1000):
    null_samples = norm.rvs(size=(100 * 100)).reshape(100, 100)
    null_samples = (null_samples * null_dists[exp_ids, 1]) + null_dists[exp_ids, 0]
    #means = [d.mean() for d in null_dists[exp_ids]]
    for j in range(0, 100):
        pass

#for i in range(0, 1000):
#    exp_ids = (np.random.rand(100)*10).astype(int)
#    [d.rvs(100) for d in null_dists[exp_ids]]

#norm.rvs(loc=0, scale=2, size=(1000*100*100))
print(time.time() - _time)

0.4920921325683594


In [169]:
A = np.random.rand(100, 100)
_time = time.time()
for i in range(0, 1000):
    for j in range(0, 100):
        np.median(A[j,:])
print(time.time() - _time)

3.877923011779785


In [173]:
A = np.random.rand(1000, 100)
_time = time.time()
for i in range(0, 1000):
    #for j in range(0, 100):
    #    np.argsort(A[j,:])
    np.median(A, axis=0)
print(time.time() - _time)
np.median(A, axis=1)

1.7500181198120117


array([0.48006185, 0.51874238, 0.57593079, 0.5387665 , 0.52855793,
       0.55530004, 0.53749618, 0.44874043, 0.54116966, 0.49925879,
       0.41034688, 0.57101327, 0.49419976, 0.50110597, 0.54326813,
       0.43046065, 0.46503451, 0.54066762, 0.46741092, 0.47975946,
       0.47798989, 0.51837035, 0.59179213, 0.46605546, 0.56899995,
       0.54639548, 0.50327701, 0.45322269, 0.4259883 , 0.4540886 ,
       0.50802357, 0.44958796, 0.43481861, 0.53494169, 0.54463983,
       0.49498721, 0.49338037, 0.56393304, 0.49281854, 0.51839627,
       0.44607203, 0.49495583, 0.45223486, 0.54210572, 0.44258172,
       0.38951369, 0.53803294, 0.47329324, 0.47302991, 0.40450793,
       0.48767621, 0.54066792, 0.52333409, 0.53119478, 0.52021142,
       0.47761235, 0.46213986, 0.40561596, 0.52936164, 0.55439443,
       0.53354781, 0.51552802, 0.52805715, 0.52669972, 0.55960751,
       0.50847506, 0.57317901, 0.49199393, 0.51052026, 0.54570238,
       0.46832147, 0.5406253 , 0.37669786, 0.43709217, 0.43971

In [174]:
_time = time.time()
pool = uniform.rvs(size=1000*100*100)
counter = 0
for i in range(0, 1000):
    for j in range(0, 100):
        fp = (pool[counter:(counter+100)] < 0.1).astype(bool)
        counter += 100
        if fp.sum() == 0: continue
print(time.time() - _time)

0.8897509574890137


In [175]:
peps = np.random.rand(100)

_time = time.time()
pool = uniform.rvs(size=1000*100*100)


print(time.time() - _time)

0.31109189987182617


In [178]:
np.repeat(np.array([1, 2, 3]), 3)

array([1, 1, 1, 2, 2, 2, 3, 3, 3])